In [1]:
import requests
import pandas as pd
import os
from tqdm import tqdm
import datetime

In [2]:
# ==============================
# 📌 Config
# ==============================
SAVE_DIR = "50_coins_crypto_1h_data_cryptocompare"
os.makedirs(SAVE_DIR, exist_ok=True)

START_DATE = "2020-01-01"
END_DATE = "2025-08-29"

# convert to UNIX timestamps
start_ts = int(datetime.datetime.strptime(START_DATE, "%Y-%m-%d").timestamp())
end_ts = int(datetime.datetime.strptime(END_DATE, "%Y-%m-%d").timestamp())

In [3]:
# ==============================
# 📌 50 Coins List (CryptoCompare symbols)
# ==============================
coins = [
    "BTC","ETH","BNB","SOL","XRP","ADA","DOGE","MATIC","DOT","LTC",
    "TRX","LINK","ATOM","XLM","ETC","FIL","ICP","APT","ARB","SUI",
    "OP","NEAR","VET","HBAR","AAVE","GRT","ALGO","QNT","EGLD","SAND",
    "MANA","AXS","THETA","FTM","FLOW","RUNE","KAVA","CHZ","DYDX","IMX",
    "1INCH","ENJ","CRV","ZIL","WAVES","KSM","DASH","BAT","ZRX","HOT"
]


In [4]:
# ==============================
# 📌 Function to fetch OHLCV from CryptoCompare
# ==============================
def get_cryptocompare_data(symbol, currency="USDT", limit=2000):
    url = f"https://min-api.cryptocompare.com/data/v2/histohour"
    params = {
        "fsym": symbol,
        "tsym": currency,
        "limit": limit,       # max 2000 hours per request
        "toTs": end_ts
    }
    
    all_data = []
    ts = end_ts
    while ts > start_ts:
        params["toTs"] = ts
        r = requests.get(url, params=params).json()
        
        if "Data" not in r or "Data" not in r["Data"]:
            break
        
        data = r["Data"]["Data"]
        if not data:
            break
        
        all_data.extend(data)
        ts = data[0]["time"] - 1  # move backward
    
    df = pd.DataFrame(all_data)
    if df.empty:
        return None
    
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.rename(columns={
        "time": "timestamp",
        "open": "open",
        "high": "high",
        "low": "low",
        "close": "close",
        "volumefrom": "volume_from",
        "volumeto": "volume_to"
    })
    df = df[["timestamp","open","high","low","close","volume_from","volume_to"]]
    
    return df

In [5]:
# ==============================
# 📌 Loop for All Coins
# ==============================
for coin in tqdm(coins, desc="Downloading 1H Data"):
    try:
        df = get_cryptocompare_data(coin, "USDT")
        if df is not None:
            save_path = os.path.join(SAVE_DIR, f"{coin}USDT_1h.csv")
            df.to_csv(save_path, index=False)
            print(f"✅ Saved {coin}USDT")
        else:
            print(f"⚠️ No data for {coin}")
    except Exception as e:
        print(f"❌ Error {coin}: {e}")

print("🎉 All 50 coins 1h data downloaded successfully from CryptoCompare!")

✅ Saved BTCUSDT


✅ Saved ETHUSDT


✅ Saved BNBUSDT


✅ Saved SOLUSDT


✅ Saved XRPUSDT


...

✅ Saved KSMUSDT


✅ Saved DASHUSDT


✅ Saved BATUSDT


✅ Saved ZRXUSDT


✅ Saved HOTUSDT
🎉 All 50 coins 1h data downloaded successfully from CryptoCompare!
